In [2]:
import sys
sys.path.append('..')
sys.path.append('/Users/jh7685/Documents/Projects/pysurfer')
from pysurfer.mgz_helper import map_values_as_mgz
from pysurfer.mgz_helper import load_mgzs
import os
import pandas as pd
import numpy as np
from sfp_nsdsyn import *
import warnings
from itertools import product
warnings.filterwarnings("ignore", category=UserWarning)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load spatial frequency preference (peak) maps for all subjects and average them

In [18]:
subj_list = [f'subj{i:02d}' for i in range(1, 9)]
val_list = ['mode', 'amp', 'sigma']

nsd_dir ='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset'
output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
fs_subj_dir='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'
overlay_dir=os.path.join(output_dir, 'sfp_maps/mgzs/nsdsyn')
fig_dir='/Volumes/server/Projects/sfp_nsd/derivatives/figures/sfp_anova/brain_maps/nsdsyn/ss'

In [19]:
all_mgzs = {}
for hemi, my_val in product(['lh', 'rh'], val_list):
    tmp_mgzs = []
    for subj in subj_list:
        f_path = os.path.join(overlay_dir, f'{hemi}.sub-{subj}_value-{my_val}_space-fsaverage.mgz')
        tmp_mgz = load_mgzs(f_path, fdata_only=False, squeeze=False)
        tmp_mgzs.append(tmp_mgz.get_fdata())
    k = f'{hemi}-{my_val}'
    all_mgzs[k] = np.mean(tmp_mgzs, axis=0)


## Save the averaged map

In [22]:

for hemi, my_val in product(['lh', 'rh'], val_list):
    k = f'{hemi}-{my_val}'
    f_path = os.path.join(overlay_dir, f'{hemi}.sub-subj01_value-{my_val}_space-fsaverage.mgz')
    save_path = os.path.join(overlay_dir, f'{hemi}.sub-avg_value-{my_val}_space-fsaverage.mgz')
    map_values_as_mgz(f_path, all_mgzs[k], save_path)


# Load precision_v map for all subjects and save the average

In [23]:
for hemi, my_val in product(['lh', 'rh'], ['precision']):
    tmp_mgzs = []
    for subj in subj_list:
        f_path = os.path.join(overlay_dir, f'{hemi}.sub-{subj}_value-{my_val}_space-fsaverage.mgz')
        tmp_mgz = load_mgzs(f_path, fdata_only=False, squeeze=False)
        tmp_mgzs.append(tmp_mgz.get_fdata())
    k = f'{hemi}-{my_val}'
    all_mgzs[k] = np.mean(tmp_mgzs, axis=0)
    save_path = os.path.join(overlay_dir, f'{hemi}.sub-avg_value-{my_val}_space-fsaverage.mgz')
    map_values_as_mgz(f_path, all_mgzs[k], save_path)

# Make a precision mask

In [34]:
precision_mask = {}
threshold=13
for hemi in ['lh', 'rh']:
    tmp_mgz = all_mgzs[f'{hemi}-precision']
    precision_mask[hemi] = np.greater(tmp_mgz, threshold)

# Apply the precision mask to the averaged maps

In [35]:
for hemi, my_val in product(['lh', 'rh'], val_list):
    k = f'{hemi}-{my_val}'
    all_mgzs[k][precision_mask[hemi] == False] = 0
    save_path = os.path.join(overlay_dir, f'{hemi}.sub-avg_value-{my_val}_space-fsaverage_masked.mgz')
    map_values_as_mgz(f_path, all_mgzs[k], save_path)